<div style="text-align: center;" align="center">
<img src="../files/static/images/cara_logo_text_cern.png" align="center" style="text-align: center; height: 12em;" /></div>
<p style="color: blue; font-weight: bold; font-size: x-large; text-align: center;" align="center">
Airborne Transmission of SARS-CoV-2
</p>
<p style="text-align: center;">
Please see the <a href="https://cara.web.cern.ch/">CARA homepage</a> for details on the methodology, assumptions and limitations of CARA.</p>

In [ ]:
import cara.apps

app = cara.apps.ExpertApplication()
app.widget